## Entendimiento de los datos

Vamos a trabajar con el conjunto de datos Churn-Modeling descargado de ***Kaggle***. Este conjunto de datos contiene detalles de los clientes de un banco y la variable objetivo es una variable binaria que refleja el hecho de si el cliente dejó el banco (cerró su cuenta) o si continúa siendo un cliente.

Las características capturan información sociodemográfica del cliente, información de productos financieros, el comportamiento y balance de su cuenta. Las características son las siguientes:

* **CustomerId**: Id único para identificar el cliente.
* **Surname**: Apellido del cliente.
* **CreditScore**: Puntaje de crédito del cliente.
* **Geography**: País al que pertenece el cliente.
* **Gender**: Genero.
* **Age**: Edad.
* **Tenure**: Número de años que el cliente ha estado en el banco.
* **Balance**: Saldo bancario del cliente.
* **NumOfProducts**: Número de productos bancarios que utiliza el cliente.
* **HasCrCard**: Si el cliente tiene tarjeta de crédito con el banco.
* **IsActiveMember**: Si el cliente es miembro activo del banco o no.
* **EstimatedSalary**: Salario estimado en dólares.
* **Exited**: 1-Si el cliente cerró la cuenta con el banco; 0-Si el cliente es retenido.

In [22]:
import warnings
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV

warnings.filterwarnings(action="ignore")

In [23]:
churn = pd.read_csv("https://raw.githubusercontent.com/stivenlopezg/DS-ONLINE-76/master/data/churn-modeling.csv",dtype={"CustomerId": "category"})
churn.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,Yes,Yes,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,No,Yes,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,Yes,No,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,No,No,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,Yes,Yes,79084.10,0


### Preprocesamiento

In [24]:
cols_to_drop = ["CustomerId", "Surname"]

churn.drop(labels=cols_to_drop, axis="columns", inplace=True)

numerical_features = churn.select_dtypes(include="number").columns.tolist()
categorical_features = churn.select_dtypes(exclude="number").columns.tolist()

numerical_features.remove("Exited")

In [25]:
exited = churn.pop("Exited")

X_train, X_test, y_train, y_test = train_test_split(churn, exited,
                                                    test_size=0.3, random_state=42)

In [26]:
numeric_preprocessing = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())

categoric_preprocessing = make_pipeline(SimpleImputer(strategy="most_frequent"))

In [27]:
X_train.loc[:, numerical_features] = numeric_preprocessing.fit_transform(X_train[numerical_features])
X_train.loc[:, categorical_features] = categoric_preprocessing.fit_transform(X_train[categorical_features])

X_train = pd.get_dummies(data=X_train, columns=categorical_features)

X_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,HasCrCard_No,HasCrCard_Yes,IsActiveMember_No,IsActiveMember_Yes
9069,-0.344595,-0.656750,-0.342170,1.583725,0.819663,1.248986,0,0,1,1,0,0,1,0,1
2603,-0.095181,-0.466380,0.698162,1.344106,-0.903352,1.522114,0,1,0,1,0,1,0,0,1
7738,-0.947345,-0.561565,0.351385,-1.222055,0.819663,1.264394,1,0,0,0,1,1,0,1,0
1579,-0.354987,0.199916,1.044940,-0.618965,-0.903352,1.647781,0,1,0,0,1,0,1,0,1
5058,0.642668,-0.180824,1.391718,1.152808,0.819663,0.875726,1,0,0,0,1,1,0,0,1


In [28]:
X_test.loc[:, numerical_features] = numeric_preprocessing.transform(X_test[numerical_features])
X_test.loc[:, categorical_features] = categoric_preprocessing.transform(X_test[categorical_features])

X_test = pd.get_dummies(data=X_test, columns=categorical_features)

X_test.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,HasCrCard_No,HasCrCard_Yes,IsActiveMember_No,IsActiveMember_Yes
6252,-0.583617,-0.656750,-0.688948,0.324894,0.819663,-1.024156,1,0,0,0,1,1,0,1,0
4684,-0.303026,0.390286,-1.382503,-1.222055,0.819663,0.790674,1,0,0,0,1,0,1,0,1
1731,-0.531655,0.485471,-0.342170,-1.222055,0.819663,-0.733117,1,0,0,1,0,0,1,1,0
4742,-1.518919,1.913248,1.044940,0.683891,0.819663,1.212328,1,0,0,0,1,0,1,0,1
4521,-0.957737,-1.132675,0.698162,0.777369,-0.903352,0.240460,1,0,0,1,0,0,1,0,1


In [29]:
knn = KNeighborsClassifier()
dtree = DecisionTreeClassifier()

In [38]:
roc = {}

roc["knn"] = cross_val_score(estimator=knn,
                             X=X_train, y=y_train, scoring="roc_auc", cv=5)

roc["dtree"] = cross_val_score(estimator=dtree,
                             X=X_train, y=y_train, scoring="roc_auc", cv=5)


In [39]:
roc = pd.DataFrame.from_dict(roc)
roc

,knn,dtree
0,0.776979,0.679002
1,0.805645,0.681342
2,0.784893,0.700157
3,0.797746,0.690742
4,0.770708,0.699006


In [32]:
params = {
    "n_neighbors": [3, 5, 7],
    "weights": ["uniform", "distance"]
}

knn_cv = GridSearchCV(estimator=knn, param_grid=params, scoring="roc_auc", cv=5)

knn_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7],
                         'weights': ['uniform', 'distance']},
             scoring='roc_auc')

In [43]:
knn_cv.best_score_

0.8060228045607827

In [34]:
knn_cv.best_params_

{'n_neighbors': 7, 'weights': 'distance'}

In [35]:
from sklearn.metrics import roc_auc_score

In [36]:
model = knn_cv.best_estimator_

prediction = model.predict(X_test)

print(f"El AUC en test es: {roc_auc_score(y_test, prediction)}")

El AUC en test es: 0.688472398621065


In [37]:
pd.DataFrame.from_dict(knn_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.030535,0.003886,0.116079,0.019008,3,uniform,"{'n_neighbors': 3, 'weights': 'uniform'}",0.744454,0.770481,0.771194,0.765381,0.729716,0.756245,0.016446,6
1,0.030130,0.001590,0.102116,0.002167,3,distance,"{'n_neighbors': 3, 'weights': 'distance'}",0.744931,0.774352,0.771093,0.771157,0.733392,0.758985,0.016634,5
2,0.027921,0.000612,0.118292,0.002153,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.776979,0.805645,0.784893,0.797746,0.770708,0.787194,0.012909,4
3,0.032500,0.004630,0.140147,0.016664,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.778307,0.809928,0.789262,0.800843,0.774543,0.790577,0.013336,3
4,0.032721,0.004396,0.146999,0.012416,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.783587,0.823597,0.798808,0.811471,0.792855,0.802064,0.014064,2
5,0.030521,0.003546,0.165348,0.026754,7,distance,"{'n_neighbors': 7, 'weights': 'distance'}",0.787118,0.830067,0.802590,0.814480,0.795859,0.806023,0.014981,1
